In [ ]:
! pip install langchain_community tiktoken langchain-google-genai langchainhub chromadb langchain

In [11]:
import os
from dotenv import load_dotenv
load_dotenv() 
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] =  os.environ.get('LANGCHAIN_API_KEY')

In [12]:
os.environ['GEMINI_API_KEY'] = os.environ.get('GEMINI_API_KEY')

In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings 
from langchain_community.vectorstores import Chroma
import os

api_key = os.environ.get("GEMINI_API_KEY")


vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=GoogleGenerativeAIEmbeddings(model="text-embedding-004", google_api_key=api_key)
)
retriever = vectorstore.as_retriever()

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key=api_key, temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

In [14]:
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough

#RAG
template = """Answer the following quetsion based on this context:

{context}

Question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key=api_key,temperature=0)
final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents is the process where the agent breaks down large, complex tasks into smaller, simpler, and more manageable subgoals or steps. This is a key component of the planning phase, enabling efficient handling of complex tasks and enhancing model performance.\n\nIt can be achieved through techniques such as:\n*   **Chain of Thought (CoT):** Instructing the model to "think step by step" to decompose hard tasks into smaller steps.\n*   **Tree of Thoughts (ToT):** Extending CoT by exploring multiple reasoning possibilities at each step, generating multiple thoughts per step to create a tree structure.\n\nTask decomposition can be done by:\n1.  The LLM itself with simple prompting (e.g., "Steps for XYZ.", "What are the subgoals for achieving XYZ?").\n2.  Using task-specific instructions (e.g., "Write a story outline." for writing a novel).\n3.  With human inputs.'